In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import json
import pandas as pd

In [2]:
BASE_URL = "https://www.idrd.gov.co"
START_URL = "https://www.idrd.gov.co/nuestros-programas"

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; DataCollector/1.0; +https://yourdomain.example)"
}

resp = requests.get(START_URL, headers=headers, timeout=15)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")

LOCALIDADES_VALIDAS = [
    "chapinero", "suba", "usaquén", "candelaria", "teusaquillo", "fontibón", "engativá", "barrios unidos",
    "bosa", "kennedy", "tunjuelito", "puente aranda", "martires", "santa fe", "antonio nariño", "san cristóbal",
    "laureles", "unificada"
]

cards = []
for marker in soup.find_all(string=re.compile(r'Escenario:', re.IGNORECASE)):
    container = marker.find_parent()
    for _ in range(4):
        text = container.get_text(separator="\n", strip=True)
        if re.search(r'Horario[:\s]', text) and re.search(r'Direcci[oó]n[:\s]', text):
            break
        if container.parent:
            container = container.parent
    text = container.get_text(separator="\n", strip=True)
    lines = [l.strip() for l in text.splitlines() if l.strip()]

    # Nombre/título heurística (primera línea significativa)
    nombre = None
    for line in lines:
        if not re.search(r'Escenario:|Direcci[oó]n:|Horario:|D[ií]as:|Localidad:|Tel[eé]fono:|Email:|Web:|Gratuita:', line, re.IGNORECASE):
            nombre = line
            break

    def extract(pattern, text):
        m = re.search(pattern, text, re.IGNORECASE)
        return m.group(1).strip() if m else None

    # Dirección
    direccion = extract(r'Direcci[oó]n[:\s]*([^\n\r]+)', text)

    # es_gratuita SIEMPRE TRUE
    es_gratuita = True

    # Días de la actividad
    dias = extract(r'D[ií]as[:\s]*([^\n\r]+)', text)

    # Contacto FIJO
    contacto = "+57 (601) 6605400 Ext.251 y 252"

    # Escenario: extrae el escenario donde se realiza la actividad (según patrón)
    escenario = extract(r'Escenario[:\s]*([^\n\r]+)', text)

    # Localidad: realiza el matching flexible, si no coincide deja el texto original
    localidad_line = extract(r'Localidad[:\s]*([^\n\r]+)', text)
    localidad = None
    if localidad_line:
        loc_clean = localidad_line.strip().lower()
        for loc_option in LOCALIDADES_VALIDAS:
            if loc_option in loc_clean:
                localidad = loc_option
                break
        if not localidad:
            localidad = loc_clean  # Si no está en lista, lo deja igual

    # Produce solo si nombre y dirección están presentes
    if nombre and direccion:
        datos = {
            'nombre': nombre,
            'direccion': direccion,
            'es_gratuita': es_gratuita,
            'dias': dias,
            'contacto': contacto,
            'escenario': escenario,
            'localidad': localidad
        }
        cards.append(datos)

# Deduplicar por (nombre, direccion)
unique = {}
for c in cards:
    key = (c.get('nombre'), c.get('direccion'))
    unique[key] = c
result = list(unique.values())

print(json.dumps(result, ensure_ascii=False, indent=2))

if result:
    df = pd.DataFrame(result)
    df.to_csv("programas_idrd_transformado.csv", index=False, encoding="utf-8-sig")
    print("Archivo CSV guardado como 'programas_idrd_transformado.csv'")
else:
    print("No se encontraron datos útiles para guardar.")

[
  {
    "nombre": "Persona Mayor",
    "direccion": "CARRERA 1A ESTE # 7-34",
    "es_gratuita": true,
    "dias": "MARTES - MIÉRCOLES - JUEVES - VIERNES",
    "contacto": "+57 (601) 6605400 Ext.251 y 252",
    "escenario": "POLIDEPORTIVO PARQUE BARRIO EGIPTO (4625)",
    "localidad": "candelaria"
  },
  {
    "nombre": "Persona Mayor",
    "direccion": "ESQUINA SUR OCCIDENTAL DE LA CALLE 159 A BIS CON CARRERA 90",
    "es_gratuita": true,
    "dias": "MARTES",
    "contacto": "+57 (601) 6605400 Ext.251 y 252",
    "escenario": "AGRUPACIÓN DE VIVIENDA LAS MERCEDES (SALITRE)",
    "localidad": "suba"
  },
  {
    "nombre": "Persona Mayor",
    "direccion": "CALLE 181A ENTRE CARRERA 7C Y 8 COSTADO SUR-. ORIENTAL.",
    "es_gratuita": true,
    "dias": "MIÉRCOLES - VIERNES",
    "contacto": "+57 (601) 6605400 Ext.251 y 252",
    "escenario": "PARQUE CODABAS",
    "localidad": "usaquén"
  },
  {
    "nombre": "Persona Mayor",
    "direccion": "AVENIDA CARRERA. 60 # 57 - 60",
    "es_grat

In [3]:
df = pd.DataFrame(result)

# Guardar en CSV
df.to_csv("programas_idrd.csv", index=False, encoding="utf-8-sig")

print("Archivo CSV guardado como 'programas_idrd.csv'")

Archivo CSV guardado como 'programas_idrd.csv'
